CTGAN Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `CTGAN` model, including how to:

-   Create an instance of `CTGAN`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `CTGAN` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is CTGAN?
--------------

The `sdv.tabular.CTGAN` model is based on the GAN-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `CTGAN` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `ctgan` installed on your
system. If you have not done it yet, please install `ctgan` now by
executing the command `pip install sdv` in a terminal.

</div>

In [1]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


As you can see, this table contains information about students which
includes, among other things:

-   Their id and gender
-   Their grades and specializations
-   Their work experience
-   The salary that they were offered
-   The duration and dates of their placement

You will notice that there is data with the following characteristics:

-   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

Let us use `CTGAN` to learn this data and then sample synthetic data
about new students to see how well the model captures the characteristics
indicated above. In order to do this you will need to:

-   Import the `sdv.tabular.CTGAN` class and create an instance of it.
-   Call its `fit` method passing our table.
-   Call its `sample` method indicating the number of synthetic rows
    that you want to generate.

In [2]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying CTGANSynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [3]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [4]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17516,M,99.756829,56.579176,Science,51.764951,Comm&Mgmt,False,0,71.508841,Mkt&HR,54.833759,NaN,True,2020-01-16,2021-04-17,NaN
1,17386,M,80.502486,55.166002,Science,68.856263,Comm&Mgmt,True,1,49.236091,Mkt&Fin,56.255051,28737.955475,False,2020-01-30,NaT,NaN
2,17332,F,92.963014,54.696134,Science,59.881642,Others,False,1,60.833303,Mkt&Fin,53.213182,28724.704355,False,NaT,2020-11-07,3.0
3,17383,F,80.774613,44.872587,Commerce,59.818147,Comm&Mgmt,False,1,50.411259,Mkt&HR,51.199924,30919.082675,True,2020-02-01,NaT,3.0
4,17540,F,85.576470,57.449695,Commerce,62.138414,Comm&Mgmt,True,0,65.819434,Mkt&Fin,53.662576,35500.179724,True,2020-08-06,2020-07-22,12.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [5]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be sent over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `CTGAN.load` method, and then you are ready to sample new data
from the loaded instance:

In [6]:
loaded = CTGAN.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `ctgan` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking at the demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [7]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [8]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
69,17418,M,69.003762,61.966991,Commerce,46.077216,Comm&Mgmt,True,1,47.797871,Mkt&Fin,64.712065,24561.411872,False,2020-01-05,2020-09-11,NaN
71,17418,M,78.319853,62.040344,Science,54.407473,Sci&Tech,True,0,48.748390,Mkt&HR,62.825392,30598.434720,False,2020-01-06,NaT,12.0
160,17418,F,93.161144,59.673213,Science,75.864458,Sci&Tech,True,1,53.784365,Mkt&Fin,65.708832,31662.472170,True,2020-07-13,NaT,6.0
185,17418,M,98.001768,52.217751,Science,54.273460,Comm&Mgmt,False,1,63.262103,Mkt&HR,71.092165,64949.266243,True,NaT,NaT,6.0


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [9]:
model = CTGAN(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,60.103522,89.535857,Commerce,72.077905,Sci&Tech,False,0,42.210056,Mkt&Fin,66.933841,31956.899081,True,2020-01-19,2021-02-08,3.0
1,1,F,77.761448,53.840538,Science,78.237902,Others,True,0,88.551597,Mkt&Fin,73.901616,NaN,True,2020-02-20,NaT,3.0
2,2,M,70.309601,92.237798,Science,80.940285,Comm&Mgmt,True,0,67.521675,Mkt&Fin,61.018250,28643.157851,True,NaT,NaT,3.0
3,3,M,50.736162,62.889343,Commerce,79.935960,Comm&Mgmt,False,0,44.126583,Mkt&HR,61.005412,22889.276852,True,NaT,2020-10-25,NaN
4,4,M,83.089961,59.210954,Commerce,87.256570,Comm&Mgmt,False,0,57.045416,Mkt&HR,76.357499,29234.325109,True,2019-12-23,2020-11-28,NaN


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [10]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [11]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [12]:
model = CTGAN(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"1707 James Crescent\nCoopertown, MI 29358",F,75.180327,71.687969,Commerce,82.764264,Comm&Mgmt,True,0,65.066930,Mkt&Fin,49.959062,24598.096615,True,2020-02-15,2020-07-23,3.0
1,1,"PSC 2024, Box 1677\nAPO AP 99732",F,70.560997,71.022320,Commerce,70.237244,Sci&Tech,False,1,45.459044,Mkt&Fin,74.429028,NaN,True,2020-02-04,2021-01-05,12.0
2,2,"9647 Martin Divide\nNorth William, NY 73345",M,73.789690,56.121225,Science,62.568652,Comm&Mgmt,True,0,45.898874,Mkt&HR,61.162049,30851.591962,True,2020-03-01,2020-05-01,6.0
3,3,"814 Mcclain Walk\nNew Melissashire, MT 20272",F,51.423207,50.790576,Science,55.524507,Comm&Mgmt,False,0,41.854421,Mkt&HR,58.591566,29976.223362,True,2020-08-27,2020-11-30,3.0
4,4,"9976 James Crest Apt. 125\nStevenhaven, GA 30830",M,51.649215,62.776156,Science,62.536724,Sci&Tech,False,0,54.760724,Mkt&HR,71.905970,NaN,True,2020-03-17,2020-03-27,NaN


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [13]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [14]:
model = CTGAN(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [15]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"39701 John Point\nLake Tammybury, MS 34466",F,67.251517,59.798448,Commerce,50.741761,Sci&Tech,True,0,77.125048,Mkt&Fin,65.347681,26605.053620,True,NaT,2020-07-17,6.0
1,1,"857 Rodgers Way\nWest Amanda, OH 51065",F,85.645934,98.046467,Science,63.343657,Comm&Mgmt,False,0,67.710126,Mkt&HR,56.034664,28283.459484,True,2020-02-15,2020-07-14,3.0
2,2,"298 Lopez Spring Apt. 302\nAlexanderfurt, DE 4...",M,75.611567,74.335247,Science,58.567241,Comm&Mgmt,True,0,58.152729,Mkt&HR,61.311530,30341.598387,False,NaT,2020-08-02,12.0
3,3,"027 Smith Neck\nWest Bobbyside, NY 84832",M,79.948478,68.434528,Commerce,50.708478,Comm&Mgmt,False,4,52.144690,Mkt&Fin,74.056580,27979.263273,True,2019-12-31,2020-07-06,3.0
4,4,"590 Matthew Fort\nCastanedaberg, MO 48250",F,76.577464,85.207332,Commerce,41.348405,Comm&Mgmt,False,0,62.282281,Mkt&Fin,69.157306,25394.660045,True,NaT,2020-09-10,NaN


Which means that none of the original addresses can be found in the
sampled data:

In [16]:
data_pii.address.isin(new_data_pii.address).sum()

0

Advanced Usage
--------------

Now that we have discovered the basics, let\'s go over a few more
advanced usage examples and see the different arguments that we can pass
to our `CTGAN` Model in order to customize it to our needs.

### How to modify the CTGAN Hyperparameters?

A part from the common Tabular Model arguments, `CTGAN` has a number of
additional hyperparameters that control its learning behavior and can
impact on the performance of the model, both in terms of quality of the
generated data and computational time.

-   `epochs` and `batch_size`: these arguments control the number of
    iterations that the model will perform to optimize its parameters,
    as well as the number of samples used in each step. Its default
    values are `300` and `500` respectively, and `batch_size` needs to
    always be a value which is multiple of `10`. These hyperparameters
    have a very direct effect in time the training process lasts but
    also on the performance of the data, so for new datasets, you might
    want to start by setting a low value on both of them to see how long
    the training process takes on your data and later on increase the
    number to acceptable values in order to improve the performance.
-   `log_frequency`: Whether to use log frequency of categorical levels
    in conditional sampling. It defaults to `True`. This argument affects
    how the model processes the frequencies of the categorical values that
    are used to condition the rest of the values. In some cases, changing
    it to `False` could lead to better performance.
-   `embedding_dim` (int): Size of the random sample passed to the
    Generator. Defaults to 128.
-   `generator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Residuals. A Resiudal Layer will be created for each
    one of the values provided. Defaults to (256, 256).
-   `discriminator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Discriminator Layers. A Linear Layer will be created
    for each one of the values provided. Defaults to (256, 256).
-   `generator_lr` (float): Learning rate for the generator. Defaults to 2e-4.
-   `generator_decay` (float): Generator weight decay for the Adam Optimizer.
    Defaults to 1e-6.
-   `discriminator_lr` (float): Learning rate for the discriminator.
    Defaults to 2e-4.
-   `discriminator_decay` (float): Discriminator weight decay for the Adam
    Optimizer. Defaults to 1e-6.
-   `discriminator_steps` (int): Number of discriminator updates to do for
    each generator update. From the WGAN paper: https://arxiv.org/abs/1701.07875.
    WGAN paper default is 5. Default used is 1 to match original CTGAN
    implementation.
-   `verbose`: Whether to print fit progress on stdout. Defaults to
    `False`.

<div class="alert alert-warning">

**Warning**

Notice that the value that you set on the `batch_size` argument must
always be a multiple of `10`!

</div>

As an example, we will try to fit the `CTGAN` model slightly increasing
the number of epochs, reducing the `batch_size`, adding one additional
layer to the models involved and using a smaller wright decay.

Before we start, we will evaluate the quality of the previously
generated data using the `sdv.evaluation.evaluate` function

In [17]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

0.327848426358381

Afterwards, we create a new instance of the `CTGAN` model with the
hyperparameter values that we want to use

In [18]:
model = CTGAN(
    primary_key='student_id',
    epochs=500,
    batch_size=100,
    generator_dim=(256, 256, 256),
    discriminator_dim=(256, 256, 256)
)

And fit to our data.

In [19]:
model.fit(data)

Finally, we are ready to generate new data and evaluate the results.

In [20]:
new_data = model.sample(len(data))
evaluate(new_data, data)

0.30929582145913104

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.


### Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the `CTGAN` model, which means we can generate only values that satisfy certain conditions. These conditional values can be passed to the `conditions` parameter in the `sample` method either as a dataframe or a dictionary.

In case a dictionary is passed, the model will generate as many rows as requested, all of which will satisfy the specified conditions, such as `gender = M`.

In [21]:
conditions = {
    'gender': 'M'
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,2,M,85.118087,56.525498,Science,47.951747,Comm&Mgmt,False,0,110.736541,Mkt&Fin,58.176042,17921.720342,False,NaT,NaT,3.0
1,3,M,92.841336,82.524480,Commerce,69.298079,Comm&Mgmt,True,0,83.561343,Mkt&Fin,71.252138,NaN,True,NaT,NaT,6.0
2,4,M,99.201149,50.098976,Science,66.972413,Sci&Tech,True,0,63.046716,Mkt&Fin,77.607110,NaN,False,2020-01-27,2020-08-24,3.0
3,1,M,54.737019,69.464838,Science,55.398785,Sci&Tech,True,0,80.285764,Mkt&HR,70.295816,NaN,True,NaT,NaT,12.0
4,1,M,101.626526,62.881054,Commerce,85.802212,Comm&Mgmt,True,0,58.720536,Mkt&Fin,72.929471,NaN,True,2020-01-27,2020-11-12,6.0


It's also possible to condition on multiple columns, such as `gender = M, 'experience_years': 0`.

In [22]:
conditions = {
    'gender': 'M',
    'experience_years': 0
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,79.172623,83.123841,Science,66.270385,Sci&Tech,False,0,80.942845,Mkt&HR,68.884065,29609.706876,True,2020-02-06,2020-07-26,NaN
1,1,M,75.700253,53.259019,Commerce,48.443350,Comm&Mgmt,True,0,56.806459,Mkt&HR,77.564075,20207.522125,True,2020-01-25,2020-07-01,3.0
2,3,M,68.169607,50.336893,Commerce,42.357714,Comm&Mgmt,False,0,85.170254,Mkt&HR,67.892427,NaN,False,2020-03-26,NaT,NaN
3,4,M,82.900888,90.262035,Commerce,73.829344,Comm&Mgmt,True,0,78.841678,Mkt&Fin,76.658488,25839.238717,True,2020-02-01,2020-07-25,3.0
4,0,M,53.957894,36.606547,Commerce,50.884975,Sci&Tech,True,0,77.756239,Mkt&HR,66.204437,NaN,False,NaT,NaT,NaN


`conditions` can also be passed as a dataframe. In that case, the model will generate one sample for each row of the dataframe, sorted in the same order. Since the model already knows how many samples to generate, passing it as a parameter is unnecessary. For example, if we want to generate three samples where `gender = M` and three samples with `gender = F`,  all of them with `work_experience = True`, we can do the following: 

In [23]:
import pandas as pd 

conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
    'work_experience': [True, True, True, True, True, True]
})
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,2,M,78.132697,50.589025,Commerce,54.559525,Comm&Mgmt,True,0,100.108476,Mkt&Fin,66.598352,26315.582474,True,NaT,NaT,NaN
1,3,M,88.600298,51.438239,Commerce,58.807373,Comm&Mgmt,True,1,100.296017,Mkt&Fin,53.675839,NaN,True,NaT,2020-08-19,6.0
2,0,M,72.092722,54.559476,Commerce,50.329060,Comm&Mgmt,True,0,97.959773,Mkt&HR,65.793822,NaN,False,2020-01-26,2020-09-07,NaN
3,9,F,85.839455,91.206651,Science,71.533561,Sci&Tech,True,0,117.326702,Mkt&HR,61.092154,NaN,True,NaT,NaT,3.0
4,14,F,54.273559,62.702820,Commerce,53.543486,Sci&Tech,True,0,104.725946,Mkt&HR,75.747054,NaN,True,2020-01-27,NaT,12.0
5,17,F,52.349328,56.033220,Science,50.420881,Others,True,0,69.925808,Mkt&HR,58.707479,NaN,False,NaT,2020-09-17,NaN


`CTGAN` also supports conditioning on continuous values, as long as the values are within the range of seen numbers. For example, if all the values of the dataset are within 0 and 1, `CTGAN` will not be able to set this value to 1000.

In [24]:
conditions = {
    'degree_perc': 70.0
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,14,F,56.799995,26.651511,Science,70.0,Comm&Mgmt,False,0,81.925759,Mkt&HR,76.604924,NaN,False,NaT,2020-07-01,NaN
1,0,F,79.582737,54.544922,Science,70.0,Others,False,0,69.579737,Mkt&Fin,75.329694,NaN,False,2020-01-18,NaT,NaN
2,23,F,75.816596,77.963987,Science,70.0,Comm&Mgmt,True,0,73.972334,Mkt&Fin,72.740939,35466.265840,True,2020-01-19,2020-09-02,12.0
3,24,M,81.276939,84.775927,Science,70.0,Comm&Mgmt,True,0,71.892008,Mkt&Fin,80.355059,22139.212554,True,2020-02-10,2020-11-16,3.0
4,34,M,89.300792,81.586415,Science,70.0,Comm&Mgmt,True,0,99.421239,Mkt&HR,59.113518,NaN,False,2020-02-11,2021-03-13,6.0


<div class="alert alert-info">

**Note**
Currently, conditional sampling works through a rejection sampling process, where rows are sampled repeatedly until one that satisfies the conditions is found. In case you are running into a `Could not get enough valid rows within x trials` or simply wish to optimize the results, there are three parameters that can be fine-tuned: max_rows_multiplier, max_retries and float_rtol. More information about these parameters can be found in the API section.


</div>

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
types of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.